# **Coding Project: Encryption in Python**
---

### **Description**
In this project, you will be writing code to encrypt and decrypt a message of your choice using a key generated through QKD.

<br>

# **Importing cirq the main library in our project**

In [ ]:
!pip install cirq
import cirq

**importing Random Library**

**defining the QKD function**

In [32]:


from random import choices
def QKD(num_bits):
  #Setup
  encode_gates = {0: cirq.I, 1: cirq.X}
  basis_gates = {'Z': cirq.I, 'X': cirq.H}

  qubits = cirq.NamedQubit.range(num_bits, prefix = 'q')

  #Alice Chooses Bits and Bases
  alice_key = choices([0, 1], k = num_bits)
  alice_bases = choices(['Z', 'X'], k = num_bits)

  #Alice Creates Qubits
  alice_circuit = cirq.Circuit()

  for bit in range(num_bits):

    encode_value = alice_key[bit]
    encode_gate = encode_gates[encode_value]

    basis_value = alice_bases[bit]
    basis_gate = basis_gates[basis_value]

    qubit = qubits[bit]
    alice_circuit.append(encode_gate(qubit))
    alice_circuit.append(basis_gate(qubit))

  #Bob chooses a Bases
  bob_bases = choices(['Z', 'X'], k = num_bits)

  bob_circuit = cirq.Circuit()

  for bit in range(num_bits):

    basis_value = bob_bases[bit]
    basis_gate = basis_gates[basis_value]

    qubit = qubits[bit]
    bob_circuit.append(basis_gate(qubit))

  #Bob Measures Qubits
  bob_circuit.append(cirq.measure(qubits, key = 'bob key'))

  #Bob Creates a Key
  bb84_circuit = alice_circuit + bob_circuit

  sim = cirq.Simulator()
  results = sim.run(bb84_circuit)
  bob_key = results.measurements['bob key'][0]

  final_alice_key = []
  final_bob_key = []

  #Compare Bases
  for bit in range(num_bits):

    if alice_bases[bit] == bob_bases[bit]:
      final_alice_key.append(alice_key[bit])
      final_bob_key.append(bob_key[bit])

  #Compare Half their Bits
  num_bits_to_compare = int(len(final_alice_key) * .5)
  if final_alice_key[0:num_bits_to_compare] == final_bob_key[0:num_bits_to_compare]:
    QKD.final_alice_key = final_alice_key[num_bits_to_compare:]
    final_bob_key = final_bob_key[num_bits_to_compare:]

    print('\n\nWe can use our keys!')
    print('Alice Key: ', QKD.final_alice_key)
    print('Bob Key: ', final_bob_key)
    print('key length', len(final_bob_key))

  else:
    print('\n\nEve was listening, we need to use a different channel!')


1. **Setup**:
   - We define dictionaries `encode_gates` and `basis_gates` that map bit values and basis values to corresponding quantum gates.
   - We create an array of qubits named `qubits` using `NamedQubit.range()` method, where each qubit is prefixed with 'q'.

2. **Alice Chooses Bits and Bases**:
   - Alice randomly chooses a bit string `alice_key` of length `num_bits` using `choices` function, representing her secret key.
   - Alice randomly chooses a bit string `alice_bases` of length `num_bits` using `choices` function, representing the bases she will use to encode her qubits.

3. **Alice Creates Qubits**:
   - Alice initializes a quantum circuit `alice_circuit`.
   - For each qubit, Alice selects an encoding gate and a basis gate based on the corresponding bit values from `alice_key` and `alice_bases`.
   - Alice applies the encoding gate followed by the basis gate to each qubit in her circuit.

4. **Bob Chooses Bases and Measures Qubits**:
   - Bob randomly chooses a bit string `bob_bases` of length `num_bits`, representing the bases he will use to measure the qubits sent by Alice.
   - Bob initializes a quantum circuit `bob_circuit`.
   - For each qubit, Bob selects a basis gate based on the corresponding bit value from `bob_bases`.
   - Bob applies the basis gate to each qubit in his circuit.

5. **Bob Measures Qubits**:
   - Bob performs measurements on the qubits using the bases specified in `bob_circuit`.
   - Bob appends a measurement operation to `bob_circuit` using `cirq.measure()` to measure all qubits and obtain measurement results.

6. **Bob Creates a Key**:
   - Bob combines his circuit with Alice's circuit to create a combined circuit `bb84_circuit`.
   - Bob simulates the combined circuit using a quantum simulator and obtains the measurement results.
   - Bob extracts his final key by comparing his measurement results with Alice's bases and extracting matching bits.

7. **Key Comparison**:
   - Alice and Bob compare their bases and extract a final key by discarding bits where their bases do not match.
   - They verify that they have a secure key by comparing a subset of their keys.

8. **Output**:
   - If the comparison is successful, the function prints the final Alice and Bob keys along with the key length.
   - If the comparison fails, the function prints a message indicating potential eavesdropping.

This function implements the Quantum Key Distribution (QKD) protocol based on the BB84 algorithm, allowing two parties (Alice and Bob) to establish a secure cryptographic key over an insecure channel, while detecting potential eavesdropping attempts.

### **Step 1**
You will first need to create some data we would like to send.

In [33]:
unencrypted_string = "Thank you Qubit x Qubit."

### **Step 2**
Use the `QKD` function, defined above, to create a key for your data.

In [34]:
QKD(60)



We can use our keys!
Alice Key:  [1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0]
Bob Key:  [1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0]
key length 16


### **Step 3**
---
You will need to create a function that can now ecrypt your message using your key. You may import from any python libraries you like to define this function.

In [35]:
def encrypt_message(message, key):
  """
  Encrypts a text message using XOR with a shared secret key.

  Args:
      message: The text message to encrypt (string).
      key: The shared secret key (binary string).

  Returns:
      The encrypted message (string).
  """
  # Convert message to binary string (ASCII)
  binary_message = ''.join(format(ord(char), 'b').zfill(8) for char in message)

  # Ensure key length matches message length (extend key if needed)
  key = key * (len(binary_message) // len(key)) + key[:len(binary_message) % len(key)]

  # XOR message with key
  encrypted_binary = ''.join(str(int(a) ^ int(b)) for a, b in zip(binary_message, key))

  # Convert encrypted binary to string
  encrypted_message = ''.join(chr(int(encrypted_binary[i:i+8], 2)) for i in range(0, len(encrypted_binary), 8))

  return encrypted_message

Let's break down the `encrypt_message` function step by step:

1. **Convert Message to Binary String**:
   - The function converts the input text message into a binary string representation using ASCII encoding. Each character in the message is converted to its corresponding 8-bit binary representation, padded with leading zeros if necessary, and concatenated to form the binary message.

2. **Ensure Key Length Matches Message Length**:
   - The function ensures that the length of the key matches the length of the binary message. If the key is shorter than the message, it is repeated to match the length of the message. If the key is longer than the message, it is truncated to match the length of the message.

3. **XOR Operation**:
   - The function performs a bitwise XOR (exclusive OR) operation between each bit of the binary message and the corresponding bit of the key. This operation combines the bits of the message and the key to produce the encrypted binary message.

4. **Convert Encrypted Binary to String**:
   - The encrypted binary message is converted back to a string representation by grouping the binary digits into chunks of 8 bits (1 byte) and converting each chunk into its corresponding ASCII character. The resulting characters are concatenated to form the encrypted message.

5. **Return Encrypted Message**:
   - The function returns the encrypted message as a string.


This function provides a simple form of encryption using XOR with a shared secret key. While it may not be as secure as modern encryption algorithms like AES, it demonstrates the basic principles of symmetric encryption.

### **Step 4**

You will need to create a function that can now decrypt your message using your key. You may import from any python libraries you like to define this function.

In [36]:
def decrypt_message(encrypted_message, key):
  """
  Decrypts an encrypted message using XOR with the shared secret key.

  Args:
      encrypted_message: The encrypted message (string).
      key: The shared secret key (binary string).

  Returns:
      The decrypted message (string).
  """
  # Convert encrypted message to binary string
  binary_message = ''.join(format(ord(char), 'b').zfill(8) for char in encrypted_message)

  # Ensure key length matches message length (extend key if needed)
  key = key * (len(binary_message) // len(key)) + key[:len(binary_message) % len(key)]

  # XOR encrypted message with key
  decrypted_binary = ''.join(str(int(a) ^ int(b)) for a, b in zip(binary_message, key))

  # Convert decrypted binary to string
  decrypted_message = ''.join(chr(int(decrypted_binary[i:i+8], 2)) for i in range(0, len(decrypted_binary), 8))

  return decrypted_message

Let's break down the `decrypt_message` function step by step:

1. **Convert Encrypted Message to Binary String**:
   - The function converts the input encrypted message into a binary string representation using ASCII encoding. Each character in the encrypted message is converted to its corresponding 8-bit binary representation, padded with leading zeros if necessary, and concatenated to form the binary message.

2. **Ensure Key Length Matches Message Length**:
   - Similar to encryption, the function ensures that the length of the key matches the length of the binary message. If the key is shorter than the message, it is repeated to match the length of the message. If the key is longer than the message, it is truncated to match the length of the message.

3. **XOR Operation**:
   - The function performs a bitwise XOR (exclusive OR) operation between each bit of the binary message and the corresponding bit of the key. This operation reverses the encryption process, recovering the original binary message.

4. **Convert Decrypted Binary to String**:
   - The decrypted binary message is converted back to a string representation by grouping the binary digits into chunks of 8 bits (1 byte) and converting each chunk into its corresponding ASCII character. The resulting characters are concatenated to form the decrypted message.

5. **Return Decrypted Message**:
   - The function returns the decrypted message as a string.

Here's the code with comments explaining each step:

This function provides the decryption counterpart to the `encrypt_message` function, allowing the decryption of messages encrypted using XOR with a shared secret key.

### **Step 5**
---
Write code to encrypt and decrypt your message using your key to ensure that you were successful.

In [37]:
key = QKD.final_alice_key

encrypted_message = encrypt_message(unencrypted_string, key)
print("Encrypted message:", encrypted_message)

decrypted_message = decrypt_message(encrypted_message, key)
print("Decrypted message:", decrypted_message)


Encrypted message: àÕßÔÍÁÔåÖÀÔÖ¥ÁÝ
Decrypted message: Thank you Qubit by Qubit


# End of Notebook

---
© 2024 The Coding School, All rights reserved